# Lab 12-3 many to many fixed
### hihello training 
* many to many
* fixed input sequence length

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from pprint import pprint

print(tf.__version__)

1.12.0


### Prepairing dataset
hih -> ihe, ihe -> hel, hel -> ell, ell -> llo

In [2]:
strings = 'hihello'
char_set = sorted(list(set('hihello' + 'hello')))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

In [3]:
data = []
for i in range(len(strings) -2):
    data.append(strings[i:i+3])

x_strings = data[0:4]
y_strings = data[1:5]

print(x_strings, y_strings)

['hih', 'ihe', 'hel', 'ell'] ['ihe', 'hel', 'ell', 'llo']


In [4]:
x_data = np.array(list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings)))
x_data_len = np.array(list(map(lambda strings : len(strings), x_strings))).astype(np.int32)
x_data = x_data.astype(np.int32)
y_data = list(map(lambda strings : [char2idx.get(char) for char in strings], y_strings))
y_data = np.array(list(map(lambda data : [to_categorical(token, len(char2idx))for token in data],y_data)))

# data 형태확인
print(x_data)
print(y_data)
# print(x_data_len)

[[1 2 1]
 [2 1 0]
 [1 0 3]
 [0 3 3]]
[[[0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]]

 [[1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]]

 [[0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]]]


In [5]:
# input
tr_dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
tr_dataset = tr_dataset.batch(batch_size = 2).repeat()

print(tr_dataset)

<RepeatDataset shapes: ((?, 3), (?, 3, 5)), types: (tf.int32, tf.float32)>


### Creating model

In [6]:
num_classes = len(char2idx)
hidden_dim = 10
max_sequence = 3

input_dim = len(char2idx)
output_dim = len(char2idx)
one_hot = np.eye(len(char2idx))

model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=input_dim, output_dim=output_dim,
                                 trainable=False, input_length=max_sequence,
                                 embeddings_initializer=keras.initializers.Constant(one_hot)))
model.add(keras.layers.SimpleRNN(units=hidden_dim, return_sequences=True))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=num_classes, activation='softmax')))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 5)              25        
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 3, 10)             160       
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 5)              55        
Total params: 240
Trainable params: 215
Non-trainable params: 25
_________________________________________________________________


### Training model

In [8]:
def sequence_loss(y_true, y_pred):
    time_step_loss = keras.losses.categorical_crossentropy(y_true=y_true, y_pred=y_pred)
    return tf.reduce_mean(time_step_loss)

In [9]:
# Creating a training operation
lr = .1
opt = tf.train.AdamOptimizer(learning_rate = lr)
model.compile(optimizer=opt,
              loss=sequence_loss)

In [10]:
model.fit(tr_dataset, epochs=30, steps_per_epoch=4//2, verbose=1)

Epoch 1/30
2/2 [==============================] - 0s 146ms/step - loss: 1.4475
Epoch 2/30
2/2 [==============================] - 0s 5ms/step - loss: 0.7651
Epoch 3/30
2/2 [==============================] - 0s 3ms/step - loss: 0.4172
Epoch 4/30
2/2 [==============================] - 0s 4ms/step - loss: 0.2816
Epoch 5/30
2/2 [==============================] - 0s 4ms/step - loss: 0.2037
Epoch 6/30
2/2 [==============================] - 0s 3ms/step - loss: 0.1676
Epoch 7/30
2/2 [==============================] - 0s 5ms/step - loss: 0.1513
Epoch 8/30
2/2 [==============================] - 0s 3ms/step - loss: 0.1410
Epoch 9/30
2/2 [==============================] - 0s 4ms/step - loss: 0.1315
Epoch 10/30
2/2 [==============================] - 0s 3ms/step - loss: 0.1247
Epoch 11/30
2/2 [==============================] - 0s 5ms/step - loss: 0.1228
Epoch 12/30
2/2 [==============================] - 0s 3ms/step - loss: 0.1244
Epoch 13/30
2/2 [==============================] - 0s 4ms/step - loss: 

### Checking performance

In [11]:
yhat = model.predict(x_data)
yhat = np.argmax(yhat, axis=-1)
print(yhat)

[[0 1 0]
 [1 0 3]
 [0 3 3]
 [3 3 4]]


In [12]:
ytrue = np.argmax(y_data, axis = -1)
print(ytrue)

[[2 1 0]
 [1 0 3]
 [0 3 3]
 [3 3 4]]
